<a href="https://colab.research.google.com/github/Thushan97/CURE/blob/master/cure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/Thushan97/CURE.git

Cloning into 'CURE'...
remote: Enumerating objects: 993, done.
remote: Counting objects: 100% (321/321), done.
remote: Compressing objects: 100% (278/278), done.
remote: Total 993 (delta 41), reused 296 (delta 29), pack-reused 672
Receiving objects: 100% (993/993), 79.62 MiB | 13.52 MiB/s, done.
Resolving deltas: 100% (53/53), done.


In [4]:
!mv /content/CURE/* /content/

In [5]:
# pretrain model download
!wget "https://zenodo.org/record/7030145/files/models.tar.xz?download=1" -c -O 'models.tar.xz'
!mkdir /content/data/models
!tar -xf models.tar.xz
!mv /content/models/* /content/data/models/


--2022-09-12 16:13:16--  https://zenodo.org/record/7030145/files/models.tar.xz?download=1
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1911937724 (1.8G) [application/octet-stream]
Saving to: ‘models.tar.xz’

models.tar.xz       100%[===================>]   1.78G  18.8MB/s    in 98s     

2022-09-12 16:14:56 (18.6 MB/s) - ‘models.tar.xz’ saved [1911937724/1911937724]



In [6]:
!pip install transformers==2.10.0 subword-nmt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 660 kB 8.6 MB/s 
     |████████████████████████████████| 5.6 MB 54.1 MB/s 
     |████████████████████████████████| 880 kB 50.8 MB/s 
     |████████████████████████████████| 1.3 MB 57.5 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=4e9c213f6dcfdb546763210ba4bc9610b09a8fdd7cf4f0e5a708b070bfe17601
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


---

Run gpt_conut_trainer.py file

In [1]:
import os
import sys
import json
import time
import codecs
import random
import numpy as np
import torch
import torch.nn as nn
from transformers import OpenAIGPTLMHeadModel

GPT_CONUT_TRAINER_DIR = os.path.abspath('/content')#os.path.abspath(__file__)[: os.path.abspath(__file__).rindex('/') + 1]

In [2]:
from src.models.gpt_conut import GPTCoNuTModel
from src.dataloader.dictionary import Dictionary
from src.dataloader.gpt_conut_data_loader import GPTCoNuTDataLoader

# New Section

In [3]:
# print(f'CUDA GPU availible : {torch.cuda.is_available()}')
# DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
class GPTCoNuTTrainer():
    def __init__(self, train_loader, valid_loader, dictionary, gpt_file):
        gpt_loaded = torch.load(gpt_file)
        config = gpt_loaded['config']
        gpt_model = OpenAIGPTLMHeadModel(config).cuda()
        gpt_model.load_state_dict(gpt_loaded['model'])

        self.train_loader = train_loader
        self.valid_loader = valid_loader
        self.dictionary = dictionary

        self.batch_size = 12
        self.load_size = 1200   # load 1200 samples from training data every time

        self.gpt_model = gpt_model
        self.model = None
        self.hyper_parameter = {}
        self.optimizer = None
        self.current_train_step = 0
        self.val_loss = {}

    def shuffle_dataset(self):
        indices = [i for i in range(len(self.train_loader.dataset))]
        random.shuffle(indices)
        return indices

    def train_step(self, samples):
        self.model.train()
        self.current_train_step += 1
        self.optimizer.zero_grad()

        batch = self.train_loader.dataset.collater(samples)
        if torch.cuda.is_available():
            outputs = self.model(
                batch['net_input']['src_tokens'].cuda(),
                batch['net_input']['src_with_prev_context'].cuda(),
                batch['net_input']['ctx_tokens'].cuda(),
                prev_tokens_index=batch['target_index'].cuda(),
                prev_tokens_with_context=batch['target_with_prev_context'].cuda(),
                labels=batch['target'].cuda(),
            )
        else:
            outputs = self.model(
                batch['net_input']['src_tokens'],
                batch['net_input']['src_with_prev_context'],
                batch['net_input']['ctx_tokens'],
                prev_tokens_index=batch['target_index'],
                prev_tokens_with_context=batch['target_with_prev_context'],
                labels=batch['target'],
            )
        logits, avg_attn_scores, apr_loss, lm_loss = outputs[:4]
        loss = apr_loss + 0.3 * lm_loss
        loss.mean().backward()
        nn.utils.clip_grad_norm_(self.model.parameters(), 0.5, norm_type=2)
        self.optimizer.step()
        return loss.mean().item(), apr_loss.mean().item(), lm_loss.mean().item()

    def valid_step(self, samples):
        self.model.eval()
        batch = self.valid_loader.dataset.collater(samples)
        outputs = self.model(
            batch['net_input']['src_tokens'].cuda(),
            batch['net_input']['src_with_prev_context'].cuda(),
            batch['net_input']['ctx_tokens'].cuda(),
            prev_tokens_index=batch['target_index'].cuda(),
            prev_tokens_with_context=batch['target_with_prev_context'].cuda(),
            labels=batch['target'].cuda(),
        )
        logits, avg_attn_scores, apr_loss, lm_loss = outputs[:4]
        loss = apr_loss + 0.3 * lm_loss
        return loss.mean().item(), apr_loss.mean().item(), lm_loss.mean().item(), logits

    def validate_and_save(self, model_id, save_dir):
        oom = 0
        with torch.no_grad():
            val_loss, val_fconv_loss, val_lm_loss = [], [], []
            for i in range(0, self.valid_loader.total_size, self.batch_size):
                samples = [self.valid_loader.dataset[j]
                           for j in range(i, min(len(self.valid_loader.dataset), i + self.batch_size))]
                try:
                    loss, fconv_loss, lm_loss, logits = self.valid_step(samples)
                    val_loss.append(float(loss))
                    val_fconv_loss.append(float(fconv_loss))
                    val_lm_loss.append(float(lm_loss))
                except Exception as e:
                    oom += 1

            info = 'val loss:{}, val apr_loss:{}, val lm_loss:{}, val ppl:{}, oom:{}'.format(
                round(float(np.mean(val_loss)), 6),
                round(float(np.mean(val_fconv_loss)), 6),
                round(float(np.mean(val_lm_loss)), 6),
                round(float(np.exp(np.mean(val_loss))), 6),
                oom
            )
            print(info)

            val_loss = np.mean(val_fconv_loss)
            checkpoint = {
                'model': self.model.state_dict(),
                'optimizer': self.optimizer.state_dict(),
                'current_step': self.current_train_step,
                'config': self.model.config(),
                'val_loss': val_loss,
            }
            torch.save(checkpoint, save_dir + 'gpt_conut_' + str(model_id) + '.pt')
            self.val_loss[model_id] = {
                'val_loss': val_loss,
                'hyper-parameter': str(self.hyper_parameter),
            }

        return val_loss

    def train(self, model_id, epochs, hyper_parameter, save_dir):
        self.hyper_parameter = hyper_parameter
        self.model = GPTCoNuTModel(
            self.dictionary, embed_dim=384, max_positions=1024,
            src_encoder_convolutions=self.hyper_parameter['src_encoder_convolutions'],
            ctx_encoder_convolutions=self.hyper_parameter['ctx_encoder_convolutions'],
            decoder_convolutions=self.hyper_parameter['decoder_convolutions'],
            dropout=self.hyper_parameter['dropout'], embed_model=self.gpt_model,
        ).cuda()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=6.25e-5)
        # self.model = nn.DataParallel(self.model, device_ids=device_ids)
        
        self.valid_loader.load_data(0, self.valid_loader.total_size)
        for epoch in range(epochs):
            start_time = time.time()
            for i in range(0, self.train_loader.total_size, self.load_size):
                oom = 0
                self.train_loader.load_data(i, i + self.load_size)
                indices = self.shuffle_dataset()
                train_loss, train_apr_loss, train_lm_loss = [], [], []

                start, end = 0, 0
                samples = []
                max_src, max_ctx, max_tgt = 0, 0, 0
                while end < len(self.train_loader.dataset):
                    sample = self.train_loader.dataset[indices[end]]
                    if max_ctx + len(sample['target']) >= 1023 \
                            or max_tgt + len(sample['prev_context']) >= 1023 \
                            or max_ctx + len(sample['source']) >= 1023 \
                            or max_src + len(sample['prev_context']) >= 1023 \
                            or end - start == self.batch_size:
                        try:
                            loss, apr_loss, lm_loss = self.train_step(samples)
                            train_loss.append(loss)
                            train_apr_loss.append(apr_loss)
                            train_lm_loss.append(lm_loss)
                        except Exception as e:
                            oom += 1

                        start = end
                        max_src, max_ctx, max_tgt = 0, 0, 0
                        samples = []
                        continue
                    max_src = max(max_src, len(sample['source']))
                    max_ctx = max(max_ctx, len(sample['prev_context']))
                    max_tgt = max(max_tgt, len(sample['target']))
                    end += 1
                    samples.append(sample)
                if len(samples) > 0:
                    try:
                        loss, apr_loss, lm_loss = self.train_step(samples)
                        train_loss.append(loss)
                        train_apr_loss.append(apr_loss)
                        train_lm_loss.append(lm_loss)
                    except Exception as e:
                        oom += 1

                if (i // self.load_size) % 10 == 0:
                    info = 'epoch:{}, load data:{}, lr:{}, loss:{}, apr_loss:{}, lm_loss:{}, time:{}s, oom:{}'.\
                        format(epoch + 1, i + self.load_size,
                               round(self.optimizer.param_groups[0]['lr'], 10),
                               round(float(np.mean(train_loss)), 6),
                               round(float(np.mean(train_apr_loss)), 6),
                               round(float(np.mean(train_lm_loss)), 6),
                               int(time.time() - start_time), oom
                               )
                    start_time = time.time()
                    print(str(model_id) + ' ' + info)

                if (i // self.load_size) % 100 == 0:
                    self.validate_and_save(model_id, save_dir)
        self.validate_and_save(model_id, save_dir)

In [5]:
if __name__ == '__main__':
    device_ids = [0, 1, 2, 3]
    os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
    
    vocab_file = GPT_CONUT_TRAINER_DIR + '/data/vocabulary/vocabulary.txt'
    train_file = GPT_CONUT_TRAINER_DIR + '/data/data/training_bpe.txt'
    valid_file = GPT_CONUT_TRAINER_DIR + '/data/data/validation_bpe.txt'
    gpt_file = GPT_CONUT_TRAINER_DIR + '/data/models/code_gpt.pt'

    dictionary = Dictionary(vocab_file, min_cnt=0)
    print('dictionary initialized, vocab size:{}'.format(len(dictionary)))

    train_loader = GPTCoNuTDataLoader(train_file, dictionary)
    valid_loader = GPTCoNuTDataLoader(valid_file, dictionary)
    print('data loader initialized, train size:{}, validate size:{}'.
          format(train_loader.total_size, valid_loader.total_size))

    trainer = GPTCoNuTTrainer(train_loader, valid_loader, dictionary, gpt_file)

    hyper_parameter = {
        'src_encoder_convolutions': ((192, 5),) * 1,
        'ctx_encoder_convolutions': ((384, 5),) * 1,
        'decoder_convolutions': ((192, 5),) * 1,
        'dropout': 0.1,
    }
    model_id = 1
    epochs = 5
    trainer.train(model_id, epochs, hyper_parameter, save_dir=GPT_CONUT_TRAINER_DIR + '/data/models/')

dictionary initialized, vocab size:50061
data loader initialized, train size:2000, validate size:100


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


1 epoch:1, load data:1200, lr:6.25e-05, loss:8.735597, apr_loss:6.386846, lm_loss:7.829169, time:64s, oom:29
val loss:4.874268, val apr_loss:3.183986, val lm_loss:5.634272, val ppl:130.878322, oom:0
1 epoch:2, load data:1200, lr:6.25e-05, loss:3.228787, apr_loss:2.85313, lm_loss:1.25219, time:67s, oom:32
val loss:2.211932, val apr_loss:1.991149, val lm_loss:0.735943, val ppl:9.133347, oom:0
1 epoch:3, load data:1200, lr:6.25e-05, loss:2.33018, apr_loss:2.082891, lm_loss:0.824297, time:71s, oom:31
val loss:1.608005, val apr_loss:1.399887, val lm_loss:0.693729, val ppl:4.992843, oom:0
1 epoch:4, load data:1200, lr:6.25e-05, loss:1.739904, apr_loss:1.508892, lm_loss:0.770041, time:68s, oom:31
val loss:1.339282, val apr_loss:1.138002, val lm_loss:0.670933, val ppl:3.816302, oom:0
1 epoch:5, load data:1200, lr:6.25e-05, loss:1.618984, apr_loss:1.381005, lm_loss:0.793266, time:64s, oom:32
val loss:1.210739, val apr_loss:1.009735, val lm_loss:0.670015, val ppl:3.355964, oom:0
val loss:1.17759

Run gpt_fconv_trainer.py file

In [6]:
import json
import os
import sys
import time
import codecs
import random
import numpy as np
import torch
import torch.nn as nn
from transformers import OpenAIGPTLMHeadModel

# GPT_FCONV_TRAINER_DIR = os.path.abspath(__file__)[: os.path.abspath(__file__).rindex('/') + 1]
GPT_FCONV_TRAINER_DIR = os.path.abspath('/content')

from src.models.gpt_fconv import GPTFConvModel
from src.dataloader.dictionary import Dictionary
from src.dataloader.gpt_fconv_data_loader import GPTFConvDataLoader


class GPTFConvTrainer():
    def __init__(self, train_loader, valid_loader, dictionary, gpt_file):
        gpt_loaded = torch.load(gpt_file)
        config = gpt_loaded['config']
        gpt_model = OpenAIGPTLMHeadModel(config).cuda()
        gpt_model.load_state_dict(gpt_loaded['model'])

        self.train_loader = train_loader
        self.valid_loader = valid_loader
        self.dictionary = dictionary

        self.batch_size = 12
        self.load_size = 1200

        self.gpt_model = gpt_model
        self.model = None
        self.hyper_parameter = {}
        self.hyper_parameter_set = {'{}'}
        self.optimizer = None
        self.current_train_step = 0
        self.val_loss = {}

    def shuffle_dataset(self):
        indices = [i for i in range(len(self.train_loader.dataset))]
        random.shuffle(indices)
        return indices

    def train_step(self, samples):
        self.model.train()
        self.current_train_step += 1
        self.optimizer.zero_grad()

        batch = self.train_loader.dataset.collater(samples)
        if torch.cuda.is_available():
            outputs = self.model(
                batch['net_input']['src_tokens'].cuda(),
                batch['net_input']['src_with_prev_context'].cuda(),
                prev_tokens_index=batch['target_index'].cuda(),
                prev_tokens_with_context=batch['target_with_prev_context'].cuda(),
                labels=batch['target'].cuda(),
            )

        logits, avg_attn_scores, apr_loss, lm_loss = outputs[:4]
        loss = apr_loss + 0.3 * lm_loss
        loss.mean().backward()
        nn.utils.clip_grad_norm_(self.model.parameters(), 0.5, norm_type=2)
        self.optimizer.step()
        return loss.mean().item(), apr_loss.mean().item(), lm_loss.mean().item()

    def valid_step(self, samples):
        self.model.eval()
        batch = self.valid_loader.dataset.collater(samples)
        outputs = self.model(
            batch['net_input']['src_tokens'].cuda(),
            batch['net_input']['src_with_prev_context'].cuda(),
            prev_tokens_index=batch['target_index'].cuda(),
            prev_tokens_with_context=batch['target_with_prev_context'].cuda(),
            labels=batch['target'].cuda(),
        )
        logits, avg_attn_scores, apr_loss, lm_loss = outputs[:4]
        loss = apr_loss + 0.3 * lm_loss
        return loss.mean().item(), apr_loss.mean().item(), lm_loss.mean().item(), logits

    def validate_and_save(self, model_id, save_dir):
        oom = 0
        with torch.no_grad():
            val_loss, val_fconv_loss, val_lm_loss = [], [], []
            for i in range(0, self.valid_loader.total_size, self.batch_size):
                samples = [self.valid_loader.dataset[j]
                           for j in range(i, min(len(self.valid_loader.dataset), i + self.batch_size))]
                try:
                    loss, fconv_loss, lm_loss, logits = self.valid_step(samples)
                    val_loss.append(float(loss))
                    val_fconv_loss.append(float(fconv_loss))
                    val_lm_loss.append(float(lm_loss))
                except Exception as e:
                    oom += 1

            info = 'val loss:{}, val apr_loss:{}, val lm_loss:{}, val ppl:{}, oom:{}'.format(
                round(float(np.mean(val_loss)), 6),
                round(float(np.mean(val_fconv_loss)), 6),
                round(float(np.mean(val_lm_loss)), 6),
                round(float(np.exp(np.mean(val_loss))), 6),
                oom
            )
            print(info)

            val_loss = np.mean(val_fconv_loss)
            checkpoint = {
                'model': self.model.state_dict(),
                'optimizer': self.optimizer.state_dict(),
                'current_step': self.current_train_step,
                'config': self.model.config(),
                'val_loss': val_loss,
            }
            torch.save(checkpoint, save_dir + 'gpt_fconv_' + str(model_id) + '.pt')
            self.val_loss[model_id] = {
                'val_loss': val_loss,
                'hyper-parameter': str(self.hyper_parameter),
            }
        return val_loss

    def train(self, model_id, epochs, hyper_parameter, save_dir):
        self.hyper_parameter = hyper_parameter
        self.model = GPTFConvModel(
                self.dictionary, embed_dim=384, max_positions=1024,
                encoder_convolutions=self.hyper_parameter['encoder_convolutions'],
                decoder_convolutions=self.hyper_parameter['decoder_convolutions'],
                dropout=self.hyper_parameter['dropout'], embed_model=self.gpt_model,
            ).cuda()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=6.25e-5)
        # self.model = nn.DataParallel(self.model, device_ids=device_ids)
        
        self.valid_loader.load_data(0, self.valid_loader.total_size)
        for epoch in range(epochs):
            start_time = time.time()
            for i in range(0, self.train_loader.total_size, self.load_size):
                oom = 0
                self.train_loader.load_data(i, i + self.load_size)
                indices = self.shuffle_dataset()
                train_loss, train_apr_loss, train_lm_loss = [], [], []

                start, end = 0, 0
                samples = []
                max_src, max_ctx, max_tgt = 0, 0, 0
                while end < len(self.train_loader.dataset):
                    sample = self.train_loader.dataset[indices[end]]
                    if max_ctx + len(sample['target']) >= 1023 \
                            or max_tgt + len(sample['prev_context']) >= 1023 \
                            or max_ctx + len(sample['source']) >= 1023 \
                            or max_src + len(sample['prev_context']) >= 1023 \
                            or end - start == self.batch_size:
                        try:
                            loss, apr_loss, lm_loss = self.train_step(samples)
                            train_loss.append(loss)
                            train_apr_loss.append(apr_loss)
                            train_lm_loss.append(lm_loss)
                        except Exception as e:
                            oom += 1

                        start = end
                        max_src, max_ctx, max_tgt = 0, 0, 0
                        samples = []
                        continue
                    max_src = max(max_src, len(sample['source']))
                    max_ctx = max(max_ctx, len(sample['prev_context']))
                    max_tgt = max(max_tgt, len(sample['target']))
                    end += 1
                    samples.append(sample)
                if len(samples) > 0:
                    try:
                        loss, apr_loss, lm_loss = self.train_step(samples)
                        train_loss.append(loss)
                        train_apr_loss.append(apr_loss)
                        train_lm_loss.append(lm_loss)
                    except Exception as e:
                        oom += 1

                if (i // self.load_size) % 10 == 0:
                    info = 'epoch:{}, load data:{}, lr:{}, loss:{}, apr_loss:{}, lm_loss:{}, time:{}s, oom:{}'.\
                        format(epoch + 1, i + self.load_size,
                               round(self.optimizer.param_groups[0]['lr'], 10),
                               round(float(np.mean(train_loss)), 6),
                               round(float(np.mean(train_apr_loss)), 6),
                               round(float(np.mean(train_lm_loss)), 6),
                               int(time.time() - start_time), oom
                               )
                    start_time = time.time()
                    print(str(model_id) + ' ' + info)

                if (i // self.load_size) % 100 == 0:
                    self.validate_and_save(model_id, save_dir)
        self.validate_and_save(model_id, save_dir)


if __name__ == '__main__':
    device_ids = [0, 1, 2, 3]
    os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"
    
    vocab_file = GPT_FCONV_TRAINER_DIR + '/data/vocabulary/vocabulary.txt'
    train_file = GPT_FCONV_TRAINER_DIR + '/data/data/training_bpe.txt'
    valid_file = GPT_FCONV_TRAINER_DIR + '/data/data/validation_bpe.txt'
    gpt_file = GPT_FCONV_TRAINER_DIR + '/data/models/code_gpt.pt'

    dictionary = Dictionary(vocab_file, min_cnt=0)
    print('dictionary initialized, vocab size:{}'.format(len(dictionary)))

    train_loader = GPTFConvDataLoader(train_file, dictionary)
    valid_loader = GPTFConvDataLoader(valid_file, dictionary)
    print('data loader initialized, train size:{}, validate size:{}'.
          format(train_loader.total_size, valid_loader.total_size))

    trainer = GPTFConvTrainer(train_loader, valid_loader, dictionary, gpt_file)

    hyper_parameter = {
        'encoder_convolutions': ((192, 5),) * 1,
        'decoder_convolutions': ((192, 5),) * 1,
        'dropout': 0.1,
    }
    trainer.train(1, 2, hyper_parameter, save_dir=GPT_FCONV_TRAINER_DIR + '/data/models/')


dictionary initialized, vocab size:50061
data loader initialized, train size:2000, validate size:100
1 epoch:1, load data:1200, lr:6.25e-05, loss:7.66629, apr_loss:5.715425, lm_loss:6.502883, time:58s, oom:11
val loss:3.377558, val apr_loss:2.772695, val lm_loss:2.016212, val ppl:29.299143, oom:0
1 epoch:2, load data:1200, lr:6.25e-05, loss:2.690209, apr_loss:2.43733, lm_loss:0.84293, time:58s, oom:14
val loss:1.790417, val apr_loss:1.565059, val lm_loss:0.751193, val ppl:5.99195, oom:0
val loss:1.634144, val apr_loss:1.415862, val lm_loss:0.727607, val ppl:5.125069, oom:0


Run generator.py file

In [22]:
import codecs
import torch
import sys
import os
from transformers import OpenAIGPTLMHeadModel

# GENERATOR_DIR = os.path.abspath(__file__)[: os.path.abspath(__file__).rindex('/') + 1]
GENERATOR_DIR = os.path.abspath('/content')
# sys.path.append(GENERATOR_DIR + '/models/')
# sys.path.append(GENERATOR_DIR + '/dataloader/')
# sys.path.append(GENERATOR_DIR + '/tester/')
from src.dataloader.gpt_conut_data_loader import GPTCoNuTDataLoader
from src.dataloader.gpt_fconv_data_loader import GPTFConvDataLoader
from src.dataloader.identifier_data_loader import IdentifierDataLoader
from src.dataloader.dictionary import Dictionary
from src.models.gpt_conut import GPTCoNuTModel
from src.models.gpt_fconv import GPTFConvModel
from src.tester.beamsearch import BeamSearch


class Generator():
    def __init__(self, model, dictionary, data_loader, beam_size=10):
        self.model = model
        self.dictionary = dictionary
        self.data_loader = data_loader
        self.beam_size = beam_size
        self.beamsearch = BeamSearch(model, dictionary, beam_size)
        print(self.model, beam_size)

    def generate(self, output_path):
        wp = codecs.open(output_path, 'w', 'utf-8')
        self.data_loader.load_data(0, self.data_loader.total_size)
        for i in range(self.data_loader.total_size):
            print(i, '/', self.data_loader.total_size)
            data = self.data_loader.dataset[i]
            if True:
                self.beamsearch.beam_size = self.beam_size
                sample = self.data_loader.dataset.collater([data])
                with torch.no_grad():
                    if isinstance(self.model, GPTCoNuTModel):
                        hypothesis = self.beamsearch.generate_gpt_conut(sample)
                    elif isinstance(self.model, GPTFConvModel):
                        hypothesis = self.beamsearch.generate_gpt_fconv(sample)
            # except Exception as e:
            #    print(e)
            #    continue
            id = str(sample['id'].item())
            wp.write('S-{}\t'.format(id))
            wp.write(self.dictionary.string(data['source']) + '\n')
            wp.write('T-{}\t'.format(id))
            wp.write(self.dictionary.string(data['target']) + '\n')
            for h in hypothesis:
                wp.write('H-{}\t{}\t'.format(id, str(h['final_score'])))
                wp.write(self.dictionary.string(h['hypo']) + '\n')
                wp.write('P-{}\t'.format(id))
                wp.write(' '.join(str(round(s.item(), 4)) for s in h['score']) + '\n')
        wp.close()


def generate_gpt_conut(vocab_file, model_file, input_file, identifier_txt_file, identifier_token_file, output_file, beam_size):
    dictionary = Dictionary(vocab_file, min_cnt=0)
    print(len(dictionary))
    loaded = torch.load(model_file, map_location='cpu')
    config = loaded['config']
    gpt_config = config['embed_model_config']
    gpt_config.attn_pdrop = 0
    gpt_config.embd_pdrop = 0
    gpt_config.resid_pdrop = 0
    gpt_model = OpenAIGPTLMHeadModel(gpt_config)
    model = GPTCoNuTModel(
        dictionary=dictionary, embed_dim=config['embed_dim'],
        max_positions=config['max_positions'],
        src_encoder_convolutions=config['src_encoder_convolutions'],
        ctx_encoder_convolutions=config['ctx_encoder_convolutions'],
        decoder_convolutions=config['decoder_convolutions'],
        dropout=0, embed_model=gpt_model,
    )

    model.load_state_dict(loaded['model'])
    identifier_loader = IdentifierDataLoader(
        dictionary, identifier_token_file, identifier_txt_file
    )
    data_loader = GPTCoNuTDataLoader(
        input_file, dictionary,
        identifier_loader=identifier_loader
    )
    generator = Generator(model, dictionary, data_loader, beam_size=beam_size)
    print('start generate')
    generator.generate(output_file)


def generate_gpt_fconv(vocab_file, model_file, input_file, identifier_txt_file, identifier_token_file, output_file, beam_size):
    dictionary = Dictionary(vocab_file, min_cnt=0)
    print(len(dictionary))
    loaded = torch.load(
        model_file, map_location='cpu'
    )
    config = loaded['config']
    gpt_config = config['embed_model_config']
    gpt_config.attn_pdrop = 0
    gpt_config.embd_pdrop = 0
    gpt_config.resid_pdrop = 0
    gpt_model = OpenAIGPTLMHeadModel(gpt_config)
    model = GPTFConvModel(
        dictionary=dictionary, embed_dim=config['embed_dim'],
        max_positions=config['max_positions'],
        encoder_convolutions=config['encoder_convolutions'],
        decoder_convolutions=config['decoder_convolutions'],
        dropout=0, embed_model=gpt_model,
    )
    model.load_state_dict(loaded['model'])
    identifier_loader = IdentifierDataLoader(
        dictionary, identifier_token_file, identifier_txt_file
    )
    data_loader = GPTFConvDataLoader(
        input_file, dictionary,
        identifier_loader=identifier_loader
    )
    generator = Generator(model, dictionary, data_loader, beam_size=beam_size)
    print('start generate')
    generator.generate(output_file)


if __name__ == "__main__":
    vocab_file = GENERATOR_DIR + '/data/vocabulary/vocabulary.txt'
    input_file = GENERATOR_DIR + '/candidate_patches/QuixBugs/quixbugs_bpe.txt'
    identifier_txt_file = GENERATOR_DIR + '/candidate_patches/QuixBugs/identifier.txt'
    identifier_token_file = GENERATOR_DIR + '/candidate_patches/QuixBugs/identifier.tokens'
    beam_size = 1000
    os.environ['CUDA_VISIBLE_DEVICES'] = "0"

    model_file = GENERATOR_DIR + '/data/models/gpt_fconv_1.pt'
    output_file = GENERATOR_DIR + '/data/patches/gpt_fconv_1.txt'
    generate_gpt_fconv(vocab_file, model_file, input_file, identifier_txt_file, identifier_token_file, output_file, beam_size)

    model_file = GENERATOR_DIR + '/data/models/gpt_conut_1.pt'
    output_file = GENERATOR_DIR + '/data/patches/gpt_conut_1.txt'
    generate_gpt_conut(vocab_file, model_file, input_file, identifier_txt_file, identifier_token_file, output_file, beam_size)

    


50061
GPTFConvModel(
  (embed_model): OpenAIGPTLMHeadModel(
    (transformer): OpenAIGPTModel(
      (tokens_embed): Embedding(50061, 384)
      (positions_embed): Embedding(1024, 384)
      (drop): Dropout(p=0, inplace=False)
      (h): ModuleList(
        (0): Block(
          (attn): Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0, inplace=False)
            (resid_dropout): Dropout(p=0, inplace=False)
          )
          (ln_1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
          (mlp): MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (dropout): Dropout(p=0, inplace=False)
          )
          (ln_2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
        )
        (1): Block(
          (attn): Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0, inplace=False)
            (resid_dropout): Dropout(p=0, inplac

AttributeError: ignored

In [23]:
import sys 
sys.version

'3.7.13 (default, Apr 24 2022, 01:04:09) \n[GCC 7.5.0]'

In [27]:
import sys 
sys.version

'3.7.13 (default, Apr 24 2022, 01:04:09) \n[GCC 7.5.0]'

In [ ]:
sample_model = torch.load("/content/data/models/gpt_f")

In [14]:
!apt-get update -y

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [91.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:14 http:

In [15]:
!apt-get install python3.8

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libpython3.8-minimal libpython3.8-stdlib python3.8-minimal
Suggested packages:
  python3.8-venv binfmt-support
The following NEW packages will be installed:
  libpython3.8-minimal libpython3.8-stdlib python3.8 python3.8-minimal
0 upgraded, 4 newly installed, 0 to remove and 51 not upgraded.
Need to get 4,695 kB of archives.
After this operation, 18.5 MB of additional disk space will be used.
Get:1 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main amd64 libpython3.8-minimal amd64 3.8.14-1+bionic1 [762 kB]
Get:2 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main amd64 python3.8-minimal amd64 3.8.14-1+bionic1 [1,839 kB]
Get:3 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic/main amd64 l

In [16]:
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8

update-alternatives: --install needs <link> <name> <path> <priority>

Use 'update-alternatives --help' for program usage information.


In [17]:
!python --version

Python 3.7.13


In [18]:
!update-alternatives --config python3

There are 2 choices for the alternative python3 (providing /usr/bin/python3).

  Selection    Path                Priority   Status
------------------------------------------------------------
* 0            /usr/bin/python3.7   2         auto mode
  1            /usr/bin/python3.6   1         manual mode
  2            /usr/bin/python3.7   2         manual mode

Press <enter> to keep the current choice[*], or type selection number: ^C


In [20]:
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 2

update-alternatives: using /usr/bin/python3.8 to provide /usr/bin/python3 (python3) in auto mode


In [25]:
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 1

In [26]:
!python --version

Python 3.8.14


In [30]:
import sys 
sys.version

'3.7.13 (default, Apr 24 2022, 01:04:09) \n[GCC 7.5.0]'

In [ ]:
import sys 
sys.version

In [29]:
!update-alternatives --config python3

There are 3 choices for the alternative python3 (providing /usr/bin/python3).

  Selection    Path                Priority   Status
------------------------------------------------------------
* 0            /usr/bin/python3.8   1         auto mode
  1            /usr/bin/python3.6   1         manual mode
  2            /usr/bin/python3.7   1         manual mode
  3            /usr/bin/python3.8   1         manual mode

Press <enter> to keep the current choice[*], or type selection number: 0


In [31]:
import sys 
sys.version

'3.7.13 (default, Apr 24 2022, 01:04:09) \n[GCC 7.5.0]'

In [32]:
!update-alternatives --config python3

There are 3 choices for the alternative python3 (providing /usr/bin/python3).

  Selection    Path                Priority   Status
------------------------------------------------------------
* 0            /usr/bin/python3.8   1         auto mode
  1            /usr/bin/python3.6   1         manual mode
  2            /usr/bin/python3.7   1         manual mode
  3            /usr/bin/python3.8   1         manual mode

Press <enter> to keep the current choice[*], or type selection number: 3


In [33]:
import sys 
sys.version

'3.7.13 (default, Apr 24 2022, 01:04:09) \n[GCC 7.5.0]'